In [ ]:
%run ../Evaluation-lib.ipynb

In [ ]:
import spot

# Prepare automata
 * Get the archive and extract it to get all the automata.

In [ ]:
%%bash --out out --err err
wget -O 20151016-automata.tar.xz http://xayn.heizmann.name/owncloud/index.php/s/Coq2zfVp5GJmfPb/download
tar --wildcards --strip-components=1 -xf 20151016-automata.tar.xz *.hoa
rm 20151016-automata.tar.xz

 * Ultimate currently produces .hoa files with wrong `Alias` lines. FIX

In [ ]:
!sed -i "/AP/,/Acceptance/ s/p\([0-9]\+\)/\1/g" complement/* semideterministic/*

# Generate Makefile
We create additional three subdirctories:
 * `simplified` -- stores the simplified automata
 * `ltl2dstar` -- stores the complements built by `ltl2dstar`
 * `products` -- stores all products needed to test correctness
 
Each subdirectory has its own `Makefile` and the top-level `Makefile` only runs them recursively and set proper dependencies. We use the simplified (by Spot's `autfilt`) automata for computing complement by `ltl2dstar` in order to achieve results for more complements.

The process of creating `Makefile` for a subdirectory (`target`) is always splitted into two halves:
1. `target` : `files`
2. rules for each file

## Simplifications

In [ ]:
def simplify_rule(aut,degree='low',timeout='5m'):
    rule='{0} : {1}\n'.format(get_simplified_name(aut,degree),\
                                 os.path.join(os.pardir,aut))
    rule+='\t{1} || touch {0}\n'.format(get_simplified_name(aut,degree),
            get_simplify_command(os.path.join(os.pardir,aut),\
            get_simplified_name(aut,degree),degree,timeout))
    return rule

In [ ]:
def simplified_target(degree='low'):
    simp_paths = []
    for case in get_cases():
        simp_paths.extend([get_simplified_name(aut,degree) for aut in get_complements(case)])
    return 'simp_{0}: {1}\n\t\n'.format(degree,' '.join(simp_paths))

## Correctness definitions

In order to verify whether $C$ is a complement of $A$ we have to check:

1. $L(A)\cap L(C) = \emptyset$
2. $\overline{L(A)}\cap \overline{L(C)} = \emptyset$

The first is easily done by `autfilt`, for the other we need the complements of $A$ and $C$ performed by LTL2DSTAR. Since LTL2DSTAR works with propositional alphabet and we work with letter alphabet, we have to adeal with this trouble. It can be done by making a product with **validate automaton**, which enforces that exactly one AP is valid at all times.

The workflow is that we create the complements of all automata, and the corresponding products by `autfilt` parallely by using Makefile. We use the simplified automata so that more complements can be computed.

### Ltl2dstar complements

In [ ]:
def ltl2dstar_target():
    automata = []
    for case in get_cases():
        automata.append(get_orig_automaton(case))
        automata.extend(get_complements(case))
    return 'ltl2dstar: {}\n\t\n'.format(' '.join([get_ltl2dstar_name(aut) for aut in automata]))

def ltl2dstar_rule(aut,timeout='5m'):
    comp_filename = get_ltl2dstar_name(aut)
    if get_algorithm_name(aut) == 'input':
        source = par(aut)
    else:
        source = par(get_simplified_path(aut,'high'))
    cmd = 'timeout {} ltl2dstar -B -H --complement-input=yes {} - | \
autfilt -H --merge-transitions > {}'.format(timeout,source,comp_filename)
    return '{0}: {1}\n\t{2} || touch {0}\n'.format(comp_filename,source,cmd)

### Commands for validate automata

In [ ]:
def get_aps_for_case(case):
    a = spot.automaton(get_orig_automaton(case))
    return [str(ap) for ap in a.ap()]

In [ ]:
def create_validate_automaton_cmd(case,timeout='5m'):
    aps = get_aps_for_case(case)
    return 'timeout {1} ltl2tgba -H -f \'G({0})\''.format("|".join(aps),timeout) +\
    ' | timeout {1} autfilt -H --exclusive-ap={0}'.format(",".join(aps),timeout)

### Commands for the `products` target in `Makefile`

In [ ]:
def get_products_target():
    complements = []
    for case in get_cases():
        complements.extend(get_complements(case))
    prods = ['prod_{}'.format(os.path.basename(c)) for c in complements]
    comp_prods = ['comp_prod_{}'.format(os.path.basename(c)) for c in complements]
    return 'prod: {} {}\n\t'.format(' '.join(prods),' '.join(comp_prods))

In [ ]:
def get_prod_rules(case,timeout='5m'):
    rules = '{0} : {2}\n\t{1} > {0} || touch {0}\n'.\
        format(get_validate_name(case),\
               create_validate_automaton_cmd(case),\
               par(get_orig_automaton(case)))
    for complement in get_complements(case):
        pr_complement=os.path.basename(complement)
        paths=dict()
        paths['product']='prod_{}'.format(pr_complement)
        paths['comp_product']='comp_prod_{}'.format(pr_complement)
        paths['A']=par(get_orig_automaton(case))
        paths['C']=par(complement)
        paths['comp_A']=par(get_ltl2dstar_path(get_orig_automaton(case)))
        paths['comp_C']=par(get_ltl2dstar_path(complement))
        paths['validate']=get_validate_name(case)
        paths['timeout']='timeout {}'.format(timeout)
        rules+='''
{comp_product} : {comp_A} {comp_C} {validate}
\t{timeout} autfilt --product={validate} -H {comp_A} |\
  {timeout} autfilt --product={comp_C} -H > {comp_product} ||\
  touch {comp_product}
    
{product} : {A} {C}
\t{timeout} autfilt --product={C} -H {A} > {product} || touch {product}
    '''.format(**paths)
    return rules

### Creation of `Makefile` files

In [ ]:
Makefile=open('Makefile','w')
mktext='''\
SUBDIRS = products ltl2dstar simplified

.PHONY: subdirs $(SUBDIRS)

subdirs: $(SUBDIRS)

$(SUBDIRS):
\t$(MAKE) -C $@

ltl2dstar: simplified

products: ltl2dstar

'''
print(mktext,file=Makefile)
Makefile.close()

#### `Makefile` for simplifications

In [ ]:
os.makedirs('simplified',exist_ok=True)
sm = open(os.path.join('simplified','Makefile'),'w')
print(simplified_target('high'),file=sm)
for case in get_cases():
    for aut in get_complements(case):
        print(simplify_rule(aut,'high'),file=sm)
sm.close()

#### `Makefile` for complements

In [ ]:
ltlstar_to = '5m'
os.makedirs('ltl2dstar',exist_ok=True)
lm = open(os.path.join('ltl2dstar','Makefile'),'w')
print(ltl2dstar_target(),file=lm)
for case in get_cases():
    print(ltl2dstar_rule(get_orig_automaton(case),ltlstar_to),file=lm)
    for aut in get_complements(case):
        print(ltl2dstar_rule(aut,ltlstar_to),file=lm)
lm.close()

#### `Makefile` for products

In [ ]:
os.makedirs('products',exist_ok=True)
pm = open(os.path.join('products','Makefile'),'w')
print(get_products_target(),file=pm)
for case in get_cases():
    print(get_prod_rules(case),file=pm)
pm.close()